In [2]:
!wget https://raw.githubusercontent.com/ifpb/openuai/main/openuai.rdf
from rdflib import Graph
from rdflib import RDF, FOAF, Literal, Namespace, URIRef

g = Graph()
g.parse(location="openuai.rdf", format='turtle')

OUAI = Namespace('http://openuai.ifpb.edu.br/ouai#')
DBO = Namespace('http://dbpedia.org/ontology/')
W3TIME = Namespace('http://www.w3.org/2006/time#')
DC = Namespace('http://purl.org/dc/terms/')
DBR = Namespace('http://dbpedia.org/resource/')

g.add([OUAI.professor32, RDF.type, DBO.Professor])
g.add([OUAI.professor32, OUAI.lattesURL, URIRef("http://lattes.cnpq.br/4442363726982518")])
g.add([OUAI.professor32, FOAF.name, Literal("Diego Ernesto Rosa Pessoa")])
g.add([OUAI.professor32, OUAI.hasSubarea, OUAI.subarea12]),
g.add([OUAI.professor32, OUAI.citationName, Literal("Diego Pessoa; PESSOA, DIEGO")])
g.add([OUAI.professor32, DBO.birthPlace, DBR['Campina_Grande']])


g.add([OUAI.courseClass142, OUAI.taughtBy, OUAI.professor32])

g.add([OUAI.coursesubject36, RDF.type, OUAI.CourseSubject])
g.add([OUAI.coursesubject36, OUAI.courseContent, Literal("Aborda as diferentes formas de armazenamento e organização de informações em um computador, com foco em estruturas como arrays, listas, pilhas, filas, árvores, grafos e tabelas hash.")])
g.add([OUAI.coursesubject36, OUAI.courseSemester, Literal(2)])
g.add([OUAI.coursesubject36, DC.title, Literal("Programação e Estrutura de Dados")])
g.add([OUAI.coursesubject36, W3TIME.hours, Literal(50.0)])
g.add([OUAI.coursesubject36, OUAI.hasBibliography, OUAI.bibliography35])
g.add([OUAI.coursesubject36, OUAI.hasBibliography, OUAI.bibliography39])
g.add([OUAI.coursesubject36, OUAI.hasBibliography, OUAI.bibliography40])

g.add([OUAI.courseClass302, RDF.type, OUAI.CourseClass])
g.add([OUAI.courseClass302, OUAI.hasCourseSubject, OUAI.coursesubject36])
g.add([OUAI.courseClass302, OUAI.classReference, Literal(20231)])
g.add([OUAI.courseClass302, OUAI.taughtBy, OUAI.professor32])

## Adicionando local de nascimento aos professores existentes
for (suj, pred, obj) in g:
    if obj in DBO.Professor:
        g.add([suj, DBO.birthPlace, DBR['João_Pessoa,_Paraíba']])


g.parse("https://dbpedia.org/resource/Campina_Grande")
g.parse("https://dbpedia.org/resource/João_Pessoa,_Paraíba")
g.parse("https://dbpedia.org/resource/Paraíba")
g.parse("https://dbpedia.org/resource/Juliette_Freire")

for (suj, pred, obj) in g:
    print(suj, pred, obj)

--2023-04-13 18:33:55--  https://raw.githubusercontent.com/ifpb/openuai/main/openuai.rdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 340501 (333K) [text/plain]
Saving to: 'openuai.rdf.3'

openuai.rdf.3       100%[===================>] 332.52K  --.-KB/s    in 0.06s   

2023-04-13 18:33:56 (5.43 MB/s) - 'openuai.rdf.3' saved [340501/340501]

http://dbpedia.org/resource/Campina_Grande http://www.w3.org/2002/07/owl#sameAs http://lt.dbpedia.org/resource/Kampina_Grandė
http://openuai.ifpb.edu.br/ouai#researchproject37 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/ontology/ResearchProject
http://openuai.ifpb.edu.br/ouai#researchproject25 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/ontology/ResearchProject
http://dbped

## 1. Construa e execute uma consulta SPARQL para exibir como retorno o nome e e o título dos projetos de pesquisa dos professores que nasceram na cidade de João Pessoa (https://dbpedia.org/page/Jo%C3%A3o_Pessoa,_Para%C3%ADba)

In [5]:
g.parse("https://dbpedia.org/resource/João_Pessoa,_Paraíba")

<Graph identifier=N6683fed1becf42fc9437411aa6f8747c (<class 'rdflib.graph.Graph'>)>

In [9]:
from rdflib import FOAF
import pandas as pd

cidade = "João Pessoa"

query = """
    SELECT distinct ?nome ?tituloProjetoPesquisa
    WHERE {
        ?prof dbo:birthPlace ?cidade ;
            foaf:name ?nome ;
            ouai:isCoordinatorIn ?projetoPesquisa .
        ?projetoPesquisa dc:title ?tituloProjetoPesquisa .
        ?cidade foaf:name ?cidadeNome .
        FILTER contains(?cidadeNome, ':cidade')
    }
    """.replace(":cidade", cidade)
res = g.query(query, initNs={'foaf': FOAF, 'ouai': OUAI, 'dc': DC, 'dbo': DBO})

pd.DataFrame([[r.nome, r.tituloProjetoPesquisa] for r in res], columns=['nome', 'projeto de pesquisa'])

,nome,projeto de pesquisa
0,Crishane Azevedo Freire,Geração de Metadados Descritivos de Conjuntos ...
1,Crishane Azevedo Freire,Identificação de Padrões e Vocabulários para R...
2,Luiz Carlos Rodrigues Chaves,FastBigData: uma Estratégia de Armazenamento E...
3,Denio Mariz Timoteo de Sousa,RH-TVDI - Programa Interinstitucional de Forma...
4,Denio Mariz Timoteo de Sousa,Performance Evaluation of Radio Resource Manag...
...,...,...
86,Lafayette Batista Melo,FOMENTO AO USO DAS TECNOLOGIAS DE COMUNICAÇÃO ...
87,Lafayette Batista Melo,Aforizações: o papel da interface e do discurs...
88,Leandro Cavalcanti de Almeida,Análise do Nível de Isolamento entre Máquinas ...
89,Leandro Cavalcanti de Almeida,Uma Proposta de Rede Sem Fio Segura de Baixo C...


## 2. Consuma dados da API do IBGE (https://servicodados.ibge.gov.br/api/docs/localidades#api-Municipios-estadosUFMunicipiosGet) para listar as cidades de um estado específico

In [67]:
import requests

sigla = "PB"
api_ibge_municipios_por_estado = "https://servicodados.ibge.gov.br/api/v1/localidades/estados/{}/municipios".format(sigla)
response = requests.get(api_ibge_municipios_por_estado)

pd.DataFrame(map(lambda item: item['nome'], response.json()), columns=['nome'])

,nome
0,Água Branca
1,Aguiar
2,Alagoa Grande
3,Alagoa Nova
4,Alagoinha
...,...
218,Uiraúna
219,Umbuzeiro
220,Várzea
221,Vieirópolis


## 3. Crie um Scraper para ler os dados dos docentes da página do PPGTI (https://www.ifpb.edu.br/ppgti/programa/corpo-docente) - construa um dataframe que liste o nome, linha de pesquisa, url do lattes e e-mail de cada professor.

In [66]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = "https://www.ifpb.edu.br/ppgti/programa/corpo-docente"
soup = BeautifulSoup(requests.get(url).content, "html.parser")
h4_tags = soup.find(id="parent-fieldname-text").select("h4")

profs = pd.DataFrame([{
        'Nome': h4.text.strip().replace('Dr.','').replace('Dra.', ''),
        'Linha de Pesquisa': h4.find_next("p").find_next('br').previous_sibling.text.replace("Linha de Pesquisa:","").strip(),
        'Lattes': h4.find_next("p").a["href"],
        'Email': h4.find_next("p").text.strip().split('E-mail:')[1]
        } for h4 in h4_tags if h4.contents]) ## se o elemento h4 está vazio deve ser ignorado
profs

,Nome,Linha de Pesquisa,Lattes,Email
0,Damires Yluska de Souza Fernandes,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/9411293540915384,damires@ifpb.edu.br
1,Diego Ernesto Rosa Pessoa,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/4442363726982518,diego.pessoa@ifpb.edu.br
2,Francisco Petrônio Alencar de Medeiros,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/9716270626654261,petronio@ifpb.edu.br
3,Heremita Brasileiro Lira,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/3532701873325164,heremita@ifpb.edu.br
4,Juliana Dantas Ribeiro Viana de Medeiros,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/9730254173461923,juliana.medeiros@ifpb.edu.br
5,Katyusco de Farias Santos,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/1246085373474860,katyusco.santos@ifpb.edu.br
6,Lafayette Batista Melo,Gestão e Desenvolvimento de Sistemas,http://lattes.cnpq.br/2144574905285987,lafayette.melo@ifpb.edu.br
7,Luciana Pereira Oliveira,Redes e Sistemas Distribuídos,http://lattes.cnpq.br/4044015211881197,luciana.oliveira@ifpb.edu.br
8,Paulo Ditarso Maciel Júnior,Redes e Sistemas Distribuídos,http://lattes.cnpq.br/1101383196957378,paulo.maciel@ifpb.edu.br
9,Paulo Ribeiro Lins Júnior,Redes e Sistemas Distribuídos,http://lattes.cnpq.br/2720405345746056,paulo.lins@ifpb.edu.br
